<a href="https://colab.research.google.com/github/ThanuMahee12/ayush-vision/blob/ssd/Algorithums/detection/SSD/SSDCustom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install torch torchvision

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
root="/content/drive/MyDrive/AyushVision/ssd/dataset/ayushvision_root"

In [56]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.models.detection import ssd300_vgg16
from torchvision.transforms import functional as F
from torchvision.datasets import VOCDetection
import torchvision.transforms as T

# Custom transform for resizing, normalization, etc.
class TransformWrapper:
    def __call__(self, image, target):
        image = F.to_tensor(image)
        image = F.resize(image, (300, 300))  # SSD300 requires image size 300x300
        return image, target

# Use a pre-trained SSD300 model
model = ssd300_vgg16(pretrained=True)
model = model.eval()  # Set to evaluation mode (for inference)

# Load dataset with Pascal VOC format
transform = TransformWrapper()

# Paths to the datasets
train_dataset = VOCDetection(root='/content/drive/MyDrive/AyushVision/ssd/dataset/ayushvision_root', year='2012', image_set='train', download=True, transforms=transform)
test_dataset = VOCDetection(root='/content/drive/MyDrive/AyushVision/ssd/dataset/ayushvision_root', year='2012', image_set='val', download=True, transforms=transform)
valid_dataset = VOCDetection(root='/content/drive/MyDrive/AyushVision/ssd/dataset/ayushvision_root', year='2012', image_set='trainval', download=True, transforms=transform)

# DataLoader for train and test
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# Check if the dataset is loaded properly
for images, targets in train_loader:
    print(f"Loaded batch of images: {len(images)}")
    break


Using downloaded and verified file: /content/drive/MyDrive/AyushVision/ssd/dataset/ayushvision_root/VOCtrainval_11-May-2012.tar
Extracting /content/drive/MyDrive/AyushVision/ssd/dataset/ayushvision_root/VOCtrainval_11-May-2012.tar to /content/drive/MyDrive/AyushVision/ssd/dataset/ayushvision_root
Using downloaded and verified file: /content/drive/MyDrive/AyushVision/ssd/dataset/ayushvision_root/VOCtrainval_11-May-2012.tar
Extracting /content/drive/MyDrive/AyushVision/ssd/dataset/ayushvision_root/VOCtrainval_11-May-2012.tar to /content/drive/MyDrive/AyushVision/ssd/dataset/ayushvision_root
Using downloaded and verified file: /content/drive/MyDrive/AyushVision/ssd/dataset/ayushvision_root/VOCtrainval_11-May-2012.tar
Extracting /content/drive/MyDrive/AyushVision/ssd/dataset/ayushvision_root/VOCtrainval_11-May-2012.tar to /content/drive/MyDrive/AyushVision/ssd/dataset/ayushvision_root
Loaded batch of images: 8


In [57]:
def train(model, dataloader, optimizer, device):
    model.train()
    for images, targets in dataloader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Forward pass
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        # Backprop
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
    return losses.item()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Set up optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train for several epochs
for epoch in range(10):  # 10 epochs
    loss = train(model, train_loader, optimizer, device)
    print(f"Epoch [{epoch+1}/10], Loss: {loss:.4f}")


AttributeError: 'dict' object has no attribute 'to'

In [48]:
from torchvision.ops import box_iou

def evaluate(model, dataloader, device):
    model.eval()
    all_iou = []
    with torch.no_grad():
        for images, targets in dataloader:
            images = list(img.to(device) for img in images)
            outputs = model(images)

            for i, output in enumerate(outputs):
                target_boxes = targets[i]['boxes'].to(device)
                pred_boxes = output['boxes']

                # Compute IoU for each image
                iou = box_iou(pred_boxes, target_boxes).mean().item()
                all_iou.append(iou)

    avg_iou = sum(all_iou) / len(all_iou)
    print(f"Average IoU: {avg_iou:.4f}")
    return avg_iou

# Evaluate on the test set
evaluate(model, test_loader, device)


AttributeError: 'Tensor' object has no attribute 'items'